**Precondition:** Download the *"186 Watershed boundary"* files from [NSDI Bhutan](https://nsdi.systems.gov.bt/data/Boundaries) and save them in `../../data/186_Watershed_boundary/`.

**Step 0: Check if DEM & ACC have the same CRS with 186 Watershed boundaries and convert if needed**

In [2]:
from pathlib import Path
import os
os.environ["GEOPANDAS_IO_ENGINE"] = "pyogrio"   
import geopandas as gpd
import rasterio   #This is a library for working with raster geospatial data (GeoTIFF)
import rioxarray as rxr # extension of xarray + rasterio, simplifies reading/writing GeoTIFF and reprojection to different CRS


acc_path = Path("../../data/HydroSHEDS/as_acc_Bhutan_and_buffer.tif") #ACC data from HydroSHEDS
dem_path = Path("../../data/HydroSHEDS/as_dem_Bhutan_and_buffer.tif") #DEM data from HydroSHEDS

polygon_coordinates_path = Path("../../data/186_Watershed_boundary/186 Watershed boundary.shp") #186 Watershed boundary.shp → geometry (polygon coordinates)
indexes_path = Path("../../data/186_Watershed_boundary/186 Watershed boundary.shx") #indexes for quick access
attribute_path = Path("../../data/186_Watershed_boundary/186 Watershed boundary.dbf") #attribute table (ID, other fields)
coordinate_system_path = Path("../../data/186_Watershed_boundary/186 Watershed boundary.prj") #To align Coordinate Reference System with DEM

if not (acc_path.exists() == dem_path.exists() == polygon_coordinates_path.exists() == indexes_path.exists() == attribute_path.exists() == coordinate_system_path.exists() ==True):
    print ("sushi")
    raise FileNotFoundError("One or more required files are missing. Please check the paths.")

# -----Check if DEM, ACC and 186_Watershed have the same CRS -----
ws = gpd.read_file(polygon_coordinates_path, engine="pyogrio")
             
with rasterio.open(dem_path) as r:
    dem_crs = r.crs
    dem_nodata = r.nodata

with rasterio.open(acc_path) as ra:
    acc_crs = ra.crs
    acc_nodata = ra.nodata

if not ws.crs == dem_crs==acc_crs:
    print("CRS NOT match!")
    print("WS CRS:", ws.crs)
    print("DEM CRS:", dem_crs)
    print("ACC CRS:", acc_crs)
    print("Let's bring all datasets to the same CRS:")

    # Covert DEM to the same CRS as the watershed shapefile (EPSG:3857)
    dem = rxr.open_rasterio(dem_path)
    dem_reproj = dem.rio.reproject(ws.crs)  # Reproject DEM to match watershed CRS
    dem_out = dem_path.parent /"dem_Bhutan_and_buffer_EPSG3857.tif"
    dem_reproj.rio.to_raster(dem_out)

    # Covert ACC to the same CRS as the watershed shapefile (EPSG:3857)
    acc = rxr.open_rasterio(acc_path)
    acc_reproj = acc.rio.reproject(ws.crs)
    acc_out = acc_path.parent /"acc_Bhutan_and_buffer_EPSG3857.tif"
    acc_reproj.rio.to_raster(acc_out)

    print(f"DEM saved to {dem_out}")
    print(f"ACC saved to {acc_out}")   
else:
    print("All datasets already share the same CRS.")

CRS NOT match!
WS CRS: EPSG:3857
DEM CRS: EPSG:4326
ACC CRS: EPSG:4326
Let's bring all datasets to the same CRS:
DEM saved to ../../data/HydroSHEDS/dem_Bhutan_and_buffer_EPSG3857.tif
ACC saved to ../../data/HydroSHEDS/acc_Bhutan_and_buffer_EPSG3857.tif


**Select unique watershed identifier**

In [ ]:
import geopandas as gpd
from pathlib import Path

ws_path = Path("../../data/186_Watershed_boundary/186 Watershed boundary.shp")

# load shapefile
ws = gpd.read_file(ws_path, engine="pyogrio")
print(ws.head())

# list all available columns
print("\nColumns in shapefile:", ws.columns.tolist())

**Compute zonal statistics for each watershed (ws_id)** 

In [ ]:
from pathlib import Path
import pandas as pd
import os
os.environ["GEOPANDAS_IO_ENGINE"] = "pyogrio"   
import geopandas as gpd
from rasterstats import zonal_stats #It allows to calculate statistics on raster values (for example, a DEM) within vector polygons


dem_reproj = Path("../../data/HydroSHEDS/dem_Bhutan_and_buffer_EPSG3857.tif")
acc_reproj = Path("../../data/HydroSHEDS/acc_Bhutan_and_buffer_EPSG3857.tif")
ws_path = Path("../../data/boundaries/186_watershed/186 Watershed boundary.shp")
# When reading a shapefile we specify only the .shp file, 
# but geopandas automatically also loads the companion files:
# - .shp → geometry
# - .dbf → attributes
# - .shx → index
# - .prj → CRS

ws = gpd.read_file(ws_path, engine="pyogrio")

# zonal_stats computes statistics for raster values within each vector polygon
#
# Commonly used stats keywords:
#   - "min"        → minimum value
#   - "max"        → maximum value
#   - "mean"       → arithmetic mean
#   - "median"     → median value
#   - "std"        → standard deviation
#   - "sum"        → sum of all valid cell values
#   - "count"      → number of valid cells
#   - "range"      → difference between max and min
#   - "majority"   → most frequent value
#   - "minority"   → least frequent value
#   - "unique"     → list of unique values
#   - "all"        → compute all available statistics
#   - "percentile_X" → e.g. "percentile_90" = 90th percentile
#
# nodata=-9999 tells the function to ignore raster cells with value -9999
# (these represent missing data / outside valid area in many DEM/ACC files)

# DEM zonal statistics
dem_stats = zonal_stats(ws, dem_reproj, stats=["min", "max", "mean", "median", "std"], nodata=-9999)
# ACC zonal statistics
acc_stats = zonal_stats(ws, acc_reproj, stats=["min", "max", "mean", "median", "std"], nodata=-9999)

# Convert lists of dicts to DataFrames and prefix columns
df_dem = pd.DataFrame(dem_stats).add_prefix("dem_")
df_acc = pd.DataFrame(acc_stats).add_prefix("acc_")

# Combine with ws_id
result = ws[["ws_id"]].copy()
result = pd.concat([ws["ws_id"].reset_index(drop=True), df_dem, df_acc], axis=1)

# Save to CSV
out_csv = ws_path.parent / "watershed_dem_acc_stats.csv"
num_records = len(result)
print(f"Number of records in result: {num_records}")
print (result.head())
result.to_csv(out_csv, index=False)
print(f"Statistics saved to {out_csv.resolve()}")